In [1]:
import pandas as pd
import numpy as np

# 计算当日的VWAP
def compute_vwap(df):
    df = df.copy()
    df['cum_vol'] = df['Volume'].cumsum()
    df['cum_pv'] = (df['Close'] * df['Volume']).cumsum()
    df['VWAP'] = df['cum_pv'] / df['cum_vol']
    return df

# 模拟单个交易日的交易逻辑
def simulate_day(day_df, allowed_times):
    """
    day_df: 当日所有分钟数据，要求已经包含了以下列：
         'Time'：格式如 '09:30'
         'Close'：收盘价格
         'UpperBound'，'LowerBound'：对应时间的当前边界
         'VWAP'：当日累计VWAP（随着时间变化更新）
    allowed_times: 列表，指允许触发交易的时刻（例如每半小时）
    
    返回当日交易记录列表，每笔记录包含 entry_time, exit_time, side, entry_price, exit_price, pnl
    """
    position = 0    # 0: 空仓, 1: 多仓, -1: 空仓
    entry_price = np.nan
    trailing_stop = np.nan  # 动态止损水平
    trade_entry_time = None
    trades = []
    
    # 对当日数据逐分钟循环（假设数据已经按时间排序）
    for idx, row in day_df.iterrows():
        current_time = row['Time']
        price = row['Close']
        upper = row['UpperBound']
        lower = row['LowerBound']
        vwap = row['VWAP']
        
        # 允许交易的时点（例如 09:30, 10:00, 10:30, …）用于开仓判断
        if position == 0 and current_time in allowed_times:
            # 当价格突破当前边界时产生开仓信号
            if price > upper:
                # 开多仓
                position = 1
                entry_price = price
                trade_entry_time = row['DateTime']
                # 对多仓，初始止损设为当前时刻的上边界与VWAP中较大者
                trailing_stop = max(upper, vwap)
                # print(f"多仓开仓 @ {current_time} price {price:.2f}")
            elif price < lower:
                # 开空仓
                position = -1
                entry_price = price
                trade_entry_time = row['DateTime']
                trailing_stop = min(lower, vwap)
                # print(f"空仓开仓 @ {current_time} price {price:.2f}")
        
        # 如果已有持仓，更新VWAP及动态止损并判断平仓信号
        if position != 0:
            # 更新动态止损：多仓取两者中较大值，空仓取两者中较小值
            if position == 1:
                new_stop = max(upper, vwap)
                trailing_stop = max(trailing_stop, new_stop)  # 仅向有利方向跟踪
                # 如果价格下穿止损，则平仓
                if price < trailing_stop:
                    exit_time = row['DateTime']
                    pnl = (price - entry_price) / entry_price  # 多仓收益率
                    trades.append({
                        'entry_time': trade_entry_time,
                        'exit_time': exit_time,
                        'side': 'Long',
                        'entry_price': entry_price,
                        'exit_price': price,
                        'pnl': pnl
                    })
                    # 重置仓位
                    position = 0
                    trailing_stop = np.nan
            elif position == -1:
                new_stop = min(lower, vwap)
                trailing_stop = min(trailing_stop, new_stop)
                if price > trailing_stop:
                    exit_time = row['DateTime']
                    pnl = (entry_price - price) / entry_price  # 空仓收益率
                    trades.append({
                        'entry_time': trade_entry_time,
                        'exit_time': exit_time,
                        'side': 'Short',
                        'entry_price': entry_price,
                        'exit_price': price,
                        'pnl': pnl
                    })
                    position = 0
                    trailing_stop = np.nan
    # 到日末若还有持仓，则以最后一分钟价格平仓
    if position != 0:
        exit_time = day_df.iloc[-1]['DateTime']
        last_price = day_df.iloc[-1]['Close']
        if position == 1:
            pnl = (last_price - entry_price) / entry_price
            trades.append({
                'entry_time': trade_entry_time,
                'exit_time': exit_time,
                'side': 'Long',
                'entry_price': entry_price,
                'exit_price': last_price,
                'pnl': pnl
            })
        else:
            pnl = (entry_price - last_price) / entry_price
            trades.append({
                'entry_time': trade_entry_time,
                'exit_time': exit_time,
                'side': 'Short',
                'entry_price': entry_price,
                'exit_price': last_price,
                'pnl': pnl
            })
    return trades

# 主程序
if __name__ == '__main__':
    # 读取数据，注意根据实际情况调整日期时间格式
    df = pd.read_csv('spy.csv', parse_dates=['DateTime'])
    df.sort_values('DateTime', inplace=True)
    
    # 提取交易日和时间（格式为 HH:MM）
    df['Date'] = df['DateTime'].dt.date
    df['Time'] = df['DateTime'].dt.strftime('%H:%M')
    
    # 以当日第一笔数据作为开盘价
    df['Open_day'] = df.groupby('Date')['Open'].transform('first')
    
    # 计算从开盘价的百分比变化
    df['ret'] = df['Close'] / df['Open_day'] - 1
    
    # 构造透视表，以便计算各分钟的历史平均绝对回报（作为σ）
    pivot = df.pivot(index='Date', columns='Time', values='ret').abs()
    # 对每个时点计算过去14个交易日的均值（如果不足则取可用值）
    sigma = pivot.rolling(window=14, min_periods=1).mean()
    sigma = sigma.stack().reset_index(name='sigma')
    # 与原数据合并，根据Date和Time
    df = pd.merge(df, sigma, on=['Date', 'Time'], how='left')
    
    # 计算Noise Area边界（当前边界）
    df['UpperBound'] = df['Open_day'] * (1 + df['sigma'])
    df['LowerBound'] = df['Open_day'] * (1 - df['sigma'])
    
    # 分日计算VWAP：对每个交易日做累加计算
    df = df.groupby('Date', group_keys=False).apply(compute_vwap)
    
    # 定义允许开仓的时点（例如交易开始后半小时，每半小时信号一次）
    # 这里可根据实际市场交易时间进行调整。例如，假设数据包含市场完整时段：
    allowed_times = ['09:30', '10:00', '10:30', '11:00', '11:30', '12:00',
                     '12:30', '13:00', '13:30', '14:00', '14:30', '15:00', '15:30']
    
    all_trades = []
    # 按日分组，逐日模拟交易
    # 注意：如果数据包含不完整的日（例如仅一部分时间），可能需要做过滤
    for trade_date, day_df in df.groupby('Date'):
        # 为了确保顺序正确，可以按时间排序
        day_df = day_df.sort_values('DateTime').reset_index(drop=True)
        trades = simulate_day(day_df, allowed_times)
        # 可以附上交易日期方便统计
        for t in trades:
            t['date'] = trade_date
        all_trades.extend(trades)
    
    # 汇总交易结果
    trades_df = pd.DataFrame(all_trades)
    if trades_df.empty:
        print("当日未触发任何交易信号。")
    else:
        # 计算累计收益（这里采用简单累乘法，不考虑复利效应）
        trades_df['return_pct'] = trades_df['pnl'] * 100
        print("交易记录:")
        print(trades_df[['date', 'side', 'entry_time', 'exit_time', 'entry_price', 'exit_price', 'return_pct']])
        
        # 简单统计：累计交易收益（假设每笔交易使用全仓100%资金）
        total_return = (trades_df['pnl'] + 1).prod() - 1
        print(f"\n累计收益率: {total_return * 100:.2f}%")


/var/folders/rq/lg6_61n50bb12byh9g0vdc380000gq/T/ipykernel_92713/2514014638.py:149: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Date', group_keys=False).apply(compute_vwap)


交易记录:
           date   side          entry_time           exit_time  entry_price  \
0    2024-01-05   Long 2024-01-05 10:00:00 2024-01-05 10:15:00     468.7900   
1    2024-01-05   Long 2024-01-05 11:00:00 2024-01-05 11:18:00     470.1600   
2    2024-01-05   Long 2024-01-05 11:30:00 2024-01-05 11:31:00     469.3700   
3    2024-01-08   Long 2024-01-08 10:00:00 2024-01-08 10:01:00     469.6350   
4    2024-01-08   Long 2024-01-08 10:30:00 2024-01-08 10:31:00     470.1600   
..          ...    ...                 ...                 ...          ...   
740  2025-04-03  Short 2025-04-03 13:00:00 2025-04-03 13:00:00     544.0199   
741  2025-04-03  Short 2025-04-03 13:30:00 2025-04-03 19:59:00     542.2448   
742  2025-04-04  Short 2025-04-04 09:30:00 2025-04-04 09:30:00     524.7600   
743  2025-04-04  Short 2025-04-04 10:00:00 2025-04-04 11:04:00     522.4800   
744  2025-04-04  Short 2025-04-04 11:30:00 2025-04-04 19:59:00     517.0900   

     exit_price  return_pct  
0      469.3800

In [2]:
import pandas as pd
import numpy as np
from math import floor, sqrt

# -------------------------------
# 辅助函数
# -------------------------------
def compute_vwap(df):
    """计算当日逐分钟累计的VWAP"""
    df = df.copy()
    df['cum_vol'] = df['Volume'].cumsum()
    df['cum_pv'] = (df['Close'] * df['Volume']).cumsum()
    df['VWAP'] = df['cum_pv'] / df['cum_vol']
    return df

def simulate_day(day_df, allowed_times, position_size):
    """
    模拟单个交易日内的交易（基于当前边界和VWAP平仓逻辑）。
    
    参数：
      day_df: 当日所有分钟数据（已包含DateTime, Time, Close, UpperBound, LowerBound, VWAP）
      allowed_times: 允许触发交易的时间点列表（例如 ['09:30', '10:00', ...]）
      position_size: 当日固定的交易份额（基于资金和VIX计算得出）
    
    返回：
      trades: 当日的所有交易记录（列表），每笔记录含有：
          entry_time, exit_time, side, entry_price, exit_price, pnl（美元盈亏）
    """
    position = 0   # 0: 空仓；1: 多仓；-1: 空仓
    entry_price = np.nan
    trailing_stop = np.nan
    trade_entry_time = None
    trades = []
    
    for idx, row in day_df.iterrows():
        current_time = row['Time']
        price = row['Close']
        upper = row['UpperBound']
        lower = row['LowerBound']
        vwap = row['VWAP']
        
        # 开仓信号只在规定时点触发
        if position == 0 and current_time in allowed_times:
            if price > upper:
                # 开多仓
                position = 1
                entry_price = price
                trade_entry_time = row['DateTime']
                # 多仓止损：取上边界和VWAP中较大值
                trailing_stop = max(upper, vwap)
            elif price < lower:
                # 开空仓
                position = -1
                entry_price = price
                trade_entry_time = row['DateTime']
                # 空仓止损：取下边界和VWAP中较小值
                trailing_stop = min(lower, vwap)
        
        # 如果持仓中，更新VWAP和动态止损并判断平仓信号
        if position != 0:
            if position == 1:
                new_stop = max(upper, vwap)
                # 仅在有利方向更新（即只向上移动）
                trailing_stop = max(trailing_stop, new_stop)
                if price < trailing_stop:
                    exit_time = row['DateTime']
                    pnl = position_size * (price - entry_price)  # 多仓盈亏：差价 * 份额
                    trades.append({
                        'entry_time': trade_entry_time,
                        'exit_time': exit_time,
                        'side': 'Long',
                        'entry_price': entry_price,
                        'exit_price': price,
                        'pnl': pnl
                    })
                    position = 0
                    trailing_stop = np.nan
            elif position == -1:
                new_stop = min(lower, vwap)
                trailing_stop = min(trailing_stop, new_stop)
                if price > trailing_stop:
                    exit_time = row['DateTime']
                    pnl = position_size * (entry_price - price)  # 空仓盈亏
                    trades.append({
                        'entry_time': trade_entry_time,
                        'exit_time': exit_time,
                        'side': 'Short',
                        'entry_price': entry_price,
                        'exit_price': price,
                        'pnl': pnl
                    })
                    position = 0
                    trailing_stop = np.nan
                    
    # 如果日末仍有持仓，则以最后价格平仓
    if position != 0:
        exit_time = day_df.iloc[-1]['DateTime']
        last_price = day_df.iloc[-1]['Close']
        if position == 1:
            pnl = position_size * (last_price - entry_price)
            trades.append({
                'entry_time': trade_entry_time,
                'exit_time': exit_time,
                'side': 'Long',
                'entry_price': entry_price,
                'exit_price': last_price,
                'pnl': pnl
            })
        else:
            pnl = position_size * (entry_price - last_price)
            trades.append({
                'entry_time': trade_entry_time,
                'exit_time': exit_time,
                'side': 'Short',
                'entry_price': entry_price,
                'exit_price': last_price,
                'pnl': pnl
            })
    return trades

# -------------------------------
# 主程序：加载数据、回测、输出每月收益率
# -------------------------------
if __name__ == '__main__':
    # 读取SPY分钟数据（假设文件名为 spy.csv）
    spy_df = pd.read_csv('spy.csv', parse_dates=['DateTime'])
    spy_df.sort_values('DateTime', inplace=True)
    spy_df['Date'] = spy_df['DateTime'].dt.date
    spy_df['Time'] = spy_df['DateTime'].dt.strftime('%H:%M')
    spy_df['Open_day'] = spy_df.groupby('Date')['Open'].transform('first')
    spy_df['ret'] = spy_df['Close'] / spy_df['Open_day'] - 1
    
    # 计算每分钟历史均值（绝对回报）用于计算Noise Area边界，采用过去14日数据
    pivot = spy_df.pivot(index='Date', columns='Time', values='ret').abs()
    sigma = pivot.rolling(window=14, min_periods=1).mean()
    sigma = sigma.stack().reset_index(name='sigma')
    spy_df = pd.merge(spy_df, sigma, on=['Date', 'Time'], how='left')
    spy_df['UpperBound'] = spy_df['Open_day'] * (1 + spy_df['sigma'])
    spy_df['LowerBound'] = spy_df['Open_day'] * (1 - spy_df['sigma'])
    
    # 计算SPY的VWAP（按日分组）
    spy_df = spy_df.groupby('Date', group_keys=False).apply(compute_vwap)
    
    # 读取VIX分钟数据（假设文件名为 vix.csv）
    vix_df = pd.read_csv('vix.csv', parse_dates=['DateTime'])
    vix_df.sort_values('DateTime', inplace=True)
    vix_df['Date'] = vix_df['DateTime'].dt.date
    vix_df['Time'] = vix_df['DateTime'].dt.strftime('%H:%M')
    
    # 定义允许开仓的时间点（半小时一次，例如从09:30到15:30）
    allowed_times = ['09:30', '10:00', '10:30', '11:00', '11:30', 
                     '12:00', '12:30', '13:00', '13:30', '14:00', '14:30', '15:00', '15:30']
    
    # 初始资金
    capital = 100000.0  
    daily_results = []  # 用于记录每个交易日的收益率和资金变化
    
    # 按交易日（spy_df中的日期）回测
    unique_dates = sorted(spy_df['Date'].unique())
    
    for trade_date in unique_dates:
        day_spy = spy_df[spy_df['Date'] == trade_date].copy()
        day_spy = day_spy.sort_values('DateTime').reset_index(drop=True)
        
        # 从vix数据中选取交易日中09:30之前的最后一个VIX值
        day_vix = vix_df[vix_df['Date'] == trade_date]
        # 筛选出时间小于'09:30'的记录
        pre_open_vix = day_vix[day_vix['Time'] < '09:30']
        if pre_open_vix.empty:
            # 若无数据，则跳过当天
            print(f"{trade_date} 无可用VIX数据，跳过")
            daily_results.append({'Date': trade_date, 'capital': capital, 'daily_return': 0})
            continue
        # 取最后一条记录
        vix_value = pre_open_vix.iloc[-1]['Close']
        # 将VIX值转为日化波动率：假设VIX为百分比（13.22表示13.22%），转换为小数后除以√252
        daily_vix = (vix_value / 100) / sqrt(252)
        target_vol = 0.02  # 2%的目标日内波动率
        multiplier = min(4, target_vol / daily_vix)
        
        # 当日开盘价
        open_price = day_spy.iloc[0]['Open_day']
        # 根据资金和乘数确定当日交易仓位（份额数）
        position_size = floor(capital * multiplier / open_price)
        if position_size <= 0:
            # 如果资金太少，则无法开仓
            print(f"{trade_date} 仓位不足，跳过")
            daily_results.append({'Date': trade_date, 'capital': capital, 'daily_return': 0})
            continue
        
        # 模拟当日的交易：使用前述的日内策略函数
        trades = simulate_day(day_spy, allowed_times, position_size)
        day_pnl = sum(trade['pnl'] for trade in trades)
        # 更新当日资金：收益为累计的交易盈亏
        capital_start = capital
        capital = capital + day_pnl
        daily_return = (capital - capital_start) / capital_start
        daily_results.append({'Date': trade_date, 'capital': capital, 'daily_return': daily_return})
        print(f"{trade_date}: 仓位={position_size}, 当日盈亏=${day_pnl:.2f}, 日收益率={daily_return*100:.2f}%")
    
    # 构造每日结果DataFrame
    daily_df = pd.DataFrame(daily_results)
    daily_df['Date'] = pd.to_datetime(daily_df['Date'])
    daily_df.set_index('Date', inplace=True)
    
    # 计算每个月的累计收益率：以每月初资金为基准，计算月末资金变化比例
    monthly = daily_df.resample('M').first()[['capital']].rename(columns={'capital': 'month_start'})
    monthly['month_end'] = daily_df.resample('M').last()['capital']
    monthly['monthly_return'] = monthly['month_end'] / monthly['month_start'] - 1
    
    print("\n每个月累计收益率:")
    print(monthly[['month_start', 'month_end', 'monthly_return']])
    
    # 如果需要，也可以计算整个回测期间的累计收益率
    total_return = capital / 100000.0 - 1
    print(f"\n整个回测期间累计收益率: {total_return*100:.2f}%")


/var/folders/rq/lg6_61n50bb12byh9g0vdc380000gq/T/ipykernel_92713/818049699.py:142: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  spy_df = spy_df.groupby('Date', group_keys=False).apply(compute_vwap)


2024-01-02: 仓位=479, 当日盈亏=$0.00, 日收益率=0.00%
2024-01-03: 仓位=485, 当日盈亏=$0.00, 日收益率=0.00%
2024-01-04: 仓位=479, 当日盈亏=$0.00, 日收益率=0.00%
2024-01-05: 仓位=489, 当日盈亏=$-97.80, 日收益率=-0.10%
2024-01-08: 仓位=492, 当日盈亏=$1658.04, 日收益率=1.66%
2024-01-09: 仓位=506, 当日盈亏=$-123.97, 日收益率=-0.12%
2024-01-10: 仓位=528, 当日盈亏=$-182.16, 日收益率=-0.18%
2024-01-11: 仓位=535, 当日盈亏=$716.90, 日收益率=0.71%
2024-01-12: 仓位=543, 当日盈亏=$84.17, 日收益率=0.08%
2024-01-16: 仓位=491, 当日盈亏=$0.00, 日收益率=0.00%
2024-01-17: 仓位=467, 当日盈亏=$-65.38, 日收益率=-0.06%
2024-01-18: 仓位=479, 当日盈亏=$378.41, 日收益率=0.37%
2024-01-19: 仓位=496, 当日盈亏=$1222.64, 日收益率=1.19%
2024-01-22: 仓位=502, 当日盈亏=$0.00, 日收益率=0.00%
2024-01-23: 仓位=516, 当日盈亏=$0.00, 日收益率=0.00%
2024-01-24: 仓位=541, 当日盈亏=$-253.51, 日收益率=-0.24%
2024-01-25: 仓位=516, 当日盈亏=$-980.45, 日收益率=-0.95%
2024-01-26: 仓位=492, 当日盈亏=$42.12, 日收益率=0.04%
2024-01-29: 仓位=479, 当日盈亏=$809.51, 日收益率=0.79%
2024-01-30: 仓位=486, 当日盈亏=$0.00, 日收益率=0.00%
2024-01-31: 仓位=506, 当日盈亏=$660.33, 日收益率=0.64%
2024-02-01: 仓位=482, 当日盈亏=$2651.00, 日收益率=2.55%
2024-02-02: 仓

/var/folders/rq/lg6_61n50bb12byh9g0vdc380000gq/T/ipykernel_92713/818049699.py:207: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = daily_df.resample('M').first()[['capital']].rename(columns={'capital': 'month_start'})
/var/folders/rq/lg6_61n50bb12byh9g0vdc380000gq/T/ipykernel_92713/818049699.py:208: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly['month_end'] = daily_df.resample('M').last()['capital']
